DeutscheWelle Data Engineering Challenge

9.05.2023


## Connect via Gmail API



In [ ]:
## Script that dowloads 


In [114]:
import pandas as pd
import psycopg2
import os
import glob



In [115]:
#Script that corrects daily reports
!ls /Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/

daily_repo_filepath= "/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/"

file1='2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv'
files=glob.glob(os.path.join(daily_repo_filepath,"*.csv"))
##date


2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv
2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv
2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv
2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv
2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2022-09-21.csv
2022-09-21-23-04-26-EDT-Historical-Report-ENTRFacebook-2022-06-22--2022-09-22.csv
2022-09-23-23-04-15-EDT-Historical-Report-ENTRFacebook-2022-06-24--2022-09-24.csv


In [119]:
files

['/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-21-23-04-26-EDT-Historical-Report-ENTRFacebook-2022-06-22--2022-09-22.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2022-09-21.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-23-23-04-15-EDT-Historical-Report-ENTRFa

In [30]:
data=pd.read_csv(daily_repo_filepath+file1)

In [31]:
data

,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,...,Message,Link,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Overperforming Score
0,Onet Styl Życia,onetstylzycia,372341662829366,NEWS_SITE,PL,Najbardziej stylowy serwis w Polsce!,2012-06-05 16:57:21,558850,563725,2022-09-15 21:00:42 EDT,...,"""24. lutego musieliśmy podjąć z mężem ważna i ...",https://www.facebook.com/onetstylzycia/videos/...,NaN,NaN,"""24. lutego wyjechałam z Kijowa. 400 kilometró...",NaN,NaN,NaN,NaN,-0.89
1,Onet Wiadomości,OnetWiadomosci,56024223012,NEWS_SITE,PL,"Jeśli macie jakieś uwagi, chcecie pokazać, jak...",2009-01-13 10:54:16,533815,625545,2022-09-15 19:00:53 EDT,...,"""Pokazują metę, ogromną kasę, przez to, że nie...",https://www.facebook.com/OnetWiadomosci/videos...,NaN,NaN,"""Influencerzy pokazują swój sukces, nie pokazu...",NaN,NaN,NaN,NaN,-4.71
2,FRANCE 24 English,FRANCE24.English,100068681555451,NEWS_SITE,FR,"🌎 International News 24/7\nInfographics, behin...",2009-10-15 12:58:12,1787138,2197295,2022-09-15 17:15:12 EDT,...,Russian forces still occupy vast swathes of Uk...,https://www.facebook.com/FRANCE24.English/vide...,NaN,NaN,War in Ukraine: Citizens of Kramatorsk face di...,NaN,NaN,NaN,NaN,2.50
3,Onet Sport,onetsport,184851814521,SPORTS,PL,Jeden z najpopularniejszych serwisów sportowyc...,2009-11-25 10:54:47,733434,722375,2022-09-15 16:54:08 EDT,...,"W studio pomeczowym podsumujemy to, co wydarzy...",https://www.facebook.com/onetsport/videos/6578...,NaN,NaN,Studio po meczu Lech Poznań - Austria Wiedeń,NaN,1.115661e+14,NaN,NaN,-2.88
4,FRANCE 24 English,FRANCE24.English,100068681555451,NEWS_SITE,FR,"🌎 International News 24/7\nInfographics, behin...",2009-10-15 12:58:12,1787138,2197295,2022-09-15 16:31:26 EDT,...,🇺🇸 The #UnitedStates has taken a step towards ...,https://www.facebook.com/FRANCE24.English/vide...,NaN,NaN,United States Moves to Provide Billions In Mil...,NaN,NaN,NaN,NaN,1.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7069,ZEIT ONLINE,zeitonline,37816894428,NEWS_SITE,DE,ZEIT ONLINE auf\n- Twitter: twitter.com/zeiton...,2008-12-06 11:00:12,825704,882606,2022-06-16 02:08:48 EDT,...,Die Friedensaktivistin Angelika Claußen ist st...,https://www.facebook.com/zeitonline/videos/192...,NaN,NaN,"Waffenlieferungen: ""Es gibt Situationen, da ka...",NaN,NaN,NaN,NaN,-3.97
7070,Noizz Food,NoizzFood,975020859234684,WEBSITE,PL,Zainspiruj się naszymi przepisami i poradami n...,2016-02-09 12:37:25,1092507,1286402,2022-06-16 02:03:49 EDT,...,"Makaron carbonara z groszkiem i łososiem, MNIA...",https://www.facebook.com/NoizzFood/videos/6885...,NaN,NaN,Carbonara z łososiem i groszkiem,NaN,NaN,NaN,NaN,-1.18
7071,FRANCE 24 English,FRANCE24.English,100068681555451,NEWS_SITE,FR,"🌎 International News 24/7\nInfographics, behin...",2009-10-15 12:58:12,1746059,2157715,2022-06-16 01:00:25 EDT,...,"Soon after the Metropol tower block in Abadan,...",https://www.facebook.com/FRANCE24.English/vide...,NaN,NaN,Iranians protest government corruption and neg...,NaN,NaN,NaN,NaN,-2.30
7072,FRANCE 24,FRANCE24,153632746935,TV_CHANNEL,FR,· Twitter : https://f24.my/TWfr\n· YouTube : h...,2009-10-15 12:57:27,10283068,11946422,2022-06-16 01:00:21 EDT,...,🇰🇪 Le Kényan Ferdinand Omanyala est le sprinte...,https://www.facebook.com/FRANCE24/videos/32499...,NaN,NaN,"Sports : A la rencontre de Ferdinand Omanyala,...",NaN,NaN,NaN,NaN,5.09


In [128]:
#data column to create SQL database
for column in data.columns.values:
    print(column.replace(" ", "_") + str(type(data[column][0])).replace("class", "").replace("<", "").replace(">", "").replace("'", "").replace("numpy.int64","int").replace("numpy.float64","float").replace("str","text") +',')    

#same with list comprehension
# types=([ column.replace(" ", "_") + str(type(data[column][0])).replace("class", "").replace("<", "").replace(">", "").replace("'", "").replace("numpy.int64","int").replace("numpy.float64","float").replace("str","text") for column in data.columns.values])

Page_Name text,
User_Name text,
Facebook_Id int,
Page_Category text,
Page_Admin_Top_Country text,
Page_Description text,
Page_Created text,
Likes_at_Posting int,
Followers_at_Posting int,
Post_Created text,
Post_Created_Date text,
Post_Created_Time text,
Type text,
Total_Interactions text,
Likes int,
Comments int,
Shares int,
Love int,
Wow int,
Haha int,
Sad int,
Angry int,
Care int,
Video_Share_Status text,
Is_Video_Owner? text,
Post_Views int,
Total_Views int,
Total_Views_For_All_Crossposts int,
Video_Length text,
URL text,
Message text,
Link text,
Final_Link float,
Image_Text float,
Link_Text text,
Description float,
Sponsor_Id float,
Sponsor_Name float,
Sponsor_Category float,
Overperforming_Score_ float,


In [135]:
## Upload data to the postres database
#connect database
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()
# cur.execute('SELECT * FROM notes')
# one = cur.fetchone()
# all = cur.fetchall()

#create database
cur.execute("""
    CREATE TABLE reports1(
    id integer PRIMARY KEY,
    Page_Name text,
    User_Name text,
    Facebook_Id int,
    Page_Category text,
    Page_Admin_Top_Country text,
    Page_Description text,
    Page_Created text,
    Likes_at_Posting int,
    Followers_at_Posting int,
    Post_Created text,
    Post_Created_Date text,
    Post_Created_Time text,
    Type text,
    Total_Interactions text,
    Likes int,
    Comments int,
    Shares int,
    Love int,
    Wow int,
    Haha int,
    Sad int,
    Angry int,
    Care int,
    Video_Share_Status text,
    Is_Video_Owner text,
    Post_Views int,
    Total_Views int,
    Total_Views_For_All_Crossposts int,
    Video_Length text,
    URL text,
    Message text,
    Link text,
    Final_Link float,
    Image_Text float,
    Link_Text text,
    Description float,
    Sponsor_Id float,
    Sponsor_Name float,
    Sponsor_Category float,
    Overperforming_Score_ float

)
""")

conn.commit()


In [120]:
#create postgres database using csv using sqlalchemy
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

df = pd.read_csv(daily_repo_filepath+file1, sep=',')
# database = postgres
# user = postgres
# host = localhost
engine = create_engine('postgresql://postgres@localhost/postgres')

df.to_sql("report", engine)

698

In [48]:
##insert data from csv
#SQL string:  
# INSERT INTO users VALUES (10, "hello@dataquest.io", "Some Name", "123 Fake St.")

#psycopg2 command to insert a single line using string interpolation instead of .format()
# cur.execute("INSERT INTO users VALUES (%s, %s, %s, %s)", (10, 'hello@dataquest.io', 'Some Name', '123 Fake St.'))
# conn.commit()

#psycopg2 to insert csv (not efficient because of the loop)
# import csv
# import psycopg2
# conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
# cur = conn.cursor()
# with open('user_accounts.csv', 'r') as f:
#     reader = csv.reader(f)
#     next(reader) # Skip the header row.
#     for row in reader:
#         cur.execute(
#         "INSERT INTO users VALUES (%s, %s, %s, %s)",
#         row
#     )
# conn.commit()

#psycopg2 command to upload data to Postgre database using efficient COPY command
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()
with open(daily_repo_filepath+file1, 'r') as f:
    # Notice that we don't need the csv module.
    next(f) # Skip the header row.
    cur.copy_from(f, 'reports', sep=',')

conn.commit()

BadCopyFileFormat: extra data after last expected column
CONTEXT:  COPY reports, line 1: "Onet Styl Życia,onetstylzycia,372341662829366,NEWS_SITE,PL,Najbardziej stylowy serwis w Polsce!,201..."


In [ ]:
##SQL script to process the data and push the data to GIT
## create tables 